In [40]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error

#dataset
data = pd.read_csv(r'G:\DNS Exfiltration Dataset\UNSW-NB15 Dataset\UNSW-NB15_50split.csv', header=None)

c:\python39\lib\site-packages\IPython\core\interactiveshell.py:3398: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421262 entries, 0 to 421261
Data columns (total 49 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       421262 non-null  object 
 1   1       421262 non-null  int64  
 2   2       421262 non-null  object 
 3   3       421262 non-null  int64  
 4   4       421262 non-null  object 
 5   5       421262 non-null  object 
 6   6       421262 non-null  float64
 7   7       421262 non-null  int64  
 8   8       421262 non-null  int64  
 9   9       421262 non-null  int64  
 10  10      421262 non-null  int64  
 11  11      421262 non-null  int64  
 12  12      421262 non-null  int64  
 13  13      421262 non-null  object 
 14  14      421262 non-null  float64
 15  15      421262 non-null  float64
 16  16      421262 non-null  int64  
 17  17      421262 non-null  int64  
 18  18      421262 non-null  int64  
 19  19      421262 non-null  int64  
 20  20      421262 non-null  int64  
 21  21      42

In [42]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,175.45.176.3,63997,149.171.126.16,53,udp,INT,0.000011,118,0,254,...,NaN,1,1,1,2,1,1,1,Generic,1
1,175.45.176.3,59883,149.171.126.16,53,udp,INT,0.000010,214,0,254,...,NaN,1,1,1,2,1,1,1,Generic,1
2,175.45.176.1,40281,149.171.126.10,53,udp,INT,0.000023,188,0,254,...,NaN,1,1,1,1,1,1,1,Reconnaissance,1
3,175.45.176.3,49934,149.171.126.16,53,udp,INT,0.000011,490,0,254,...,NaN,1,1,1,2,1,1,1,Generic,1
4,175.45.176.3,33432,149.171.126.14,53,tcp,FIN,0.529554,660,268,254,...,NaN,1,1,2,2,1,1,2,DoS,1


In [43]:
x = data[[22]]


label = data[48]

print(label.shape)

(421262,)


In [44]:
#split data into training and testing
from sklearn.model_selection import train_test_split, cross_val_score

X_train, X_test, y_train, y_test = train_test_split(x, label, test_size=0.35, random_state=42)


In [45]:
chunk = int(X_train.shape[0] * 0.2)
# print(chunk)
X_train = X_train.head(chunk)
y_train = y_train.head(chunk)
# X_train.head()
# y_train.head()
print(X_train.shape[0]+X_test.shape[0])

202206


In [46]:
#scale the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [47]:
#train the model and measure time

from sklearn.naive_bayes import GaussianNB
import time

model = GaussianNB()


scores = cross_val_score(model, X_train, y_train, cv=10)

start_time = time.time()
model.fit(X_train_scaled, y_train)
training_time = time.time() - start_time
print("training time:", training_time) 

training time: 0.004006147384643555


In [48]:
print(training_time)

0.004006147384643555


In [49]:
#predict using the trained model
from sklearn.metrics import accuracy_score

start_time = time.time()
y_pred = model.predict(X_test_scaled)
training_time = time.time() - start_time
print("testing time: ", training_time) 
accuracy = accuracy_score(y_test, y_pred)

print("accuracy: \n", accuracy)

print("10-fold Cross-Validation Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

testing time:  0.012988567352294922
accuracy: 
 0.8818111528601077
10-fold Cross-Validation Accuracy: 0.88 (+/- 0.01)


In [50]:
#create the confusion matrix
from sklearn.metrics import confusion_matrix

print("confusion matrix: \n", confusion_matrix(y_test, y_pred))


confusion matrix: 
 [[56336 17199]
 [  227 73680]]


In [51]:
#how many attacks the model detected
np.count_nonzero(y_pred == True)

90879

In [52]:
#how many attacks in the testing dataset
counter = 0

for i in y_test:
    if i == True:
        counter+=1
        
print(counter)

73907
